In [98]:
# Import pandas
import pandas as pd

# Read train data
train = pd.read_csv('train.csv', index_col='Id')

# Look at the shape of the data
print('Train shape:', train.shape)

# Look at the head() of the data
train

Train shape: (15120, 55)


,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
Id,,,,,,,,,,,,,,,,,,,,,
1,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
2,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
3,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
4,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
5,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15116,2607,243,23,258,7,660,170,251,214,1282,...,0,0,0,0,0,0,0,0,0,3
15117,2603,121,19,633,195,618,249,221,91,1325,...,0,0,0,0,0,0,0,0,0,3
15118,2492,134,25,365,117,335,250,220,83,1187,...,0,0,0,0,0,0,0,0,0,3


In [99]:


# Read the test data
test = pd.read_csv('test.csv', index_col='Id')
# Print train and test columns
print('Train columns:', train.columns.tolist())
print('Test columns:', test.columns.tolist())

# Read the sample submission file
sample_submission = pd.read_csv('sampleSubmission.csv')



Train columns: ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40', 'Cover_Type']
Test columns: ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_H

In [100]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder, OneHotEncoder
X_train, X_test, y_train, y_test = train_test_split(train.drop(columns='Cover_Type'), train['Cover_Type'], test_size=0.2)
min_max_scaler = MinMaxScaler()
X_train_scaled=min_max_scaler.fit_transform(X_train)
X_test_scaled=min_max_scaler.transform(X_test)
test_scaled=min_max_scaler.transform(test)

In [101]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier()
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score
param_grid = { 
    'n_estimators': [120],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [20,23],
    'criterion' :['gini', 'entropy']
}

In [102]:
rs = RandomizedSearchCV(estimator=model, param_distributions=param_grid,
                                   )
rs.fit(X_train_scaled, y_train)
print(rs.best_params_)
best_model = rs.best_estimator_
y_pred = best_model.predict(X_test_scaled)
print('Train accuracy: ',accuracy_score(y_train, best_model.predict(X_train_scaled)),
      ' Test accuracy: ',accuracy_score(y_test, y_pred))

{'n_estimators': 120, 'max_features': 'auto', 'max_depth': 23, 'criterion': 'entropy'}
Train accuracy:  1.0  Test accuracy:  0.8647486772486772


In [103]:
y = best_model.predict(test_scaled)

In [104]:
my_submission = pd.DataFrame({'Id': test.index, 'Cover_Type': y})
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)